## sklearn_logistic_regression
Creates sklearn multiclass classifier
with hyperparameter optimalization
and feature selection

# 0. Initialization

In [1]:
# First we load the main libraries we will need

import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import sklearn as sk
import sklearn.datasets

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

rng = np.random.RandomState(42)

#data
from sklearn.model_selection import train_test_split
#Feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

print("Environment ready!")

Environment ready!


In [2]:
new_version = True

In [3]:
import emoji
import regex

def emojis_str(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return ''.join(emoji_list)

# 1. Data

In [4]:
data = pd.read_csv("tweets_final.csv") 
data.head(5)


,Unnamed: 0,Unnamed: 0.1,id_str,Screen Name,User Name,Time,Followers,num_retweets,num_likes,date,...,reply,quote,tweet_type,num_emojis,emoji_polarity,length,party_affiliation,days_since_post,time_day,day_week
0,0,0,1.210867e+18,pvdabelgie,PVDA België,2019-12-28 10:17:26,10460,5,19,2019-12-28,...,False,False,normal,0,0.0,191,pvdabelgie,5,10,5
1,1,1,1.209871e+18,pvdabelgie,PVDA België,2019-12-25 16:16:48,10460,14,30,2019-12-25,...,False,False,normal,0,0.0,87,pvdabelgie,8,16,2
2,2,2,1.207912e+18,pvdabelgie,PVDA België,2019-12-20 06:33:56,10460,13,43,2019-12-20,...,False,False,normal,0,0.0,109,pvdabelgie,13,6,4
3,3,3,1.207627e+18,pvdabelgie,PVDA België,2019-12-19 11:39:33,10460,3,4,2019-12-19,...,False,False,normal,0,0.0,65,pvdabelgie,14,11,3
4,4,4,1.206876e+18,pvdabelgie,PVDA België,2019-12-17 09:58:19,10460,21,51,2019-12-17,...,False,False,normal,0,0.0,83,pvdabelgie,16,9,1


In [5]:
from datetime import datetime
from datetime import date
from sklearn.preprocessing import MinMaxScaler
#robust = RobustScaler(quantile_range = (0.1,0.9))
scaler = MinMaxScaler()

emojis = data['tweet_text'].apply(emojis_str)
data['num_emojis'] = emojis.apply(len)
scores=emojis.apply(analyser.polarity_scores)
data['emoji_polarity'] = (scores.apply(pd.Series)['compound']+1)/2
data['reply'] = data.replied_id.ge(0, fill_value=-1)
data['quote'] = data.quoted_id.ge(0, fill_value=-1)
data['length'] =data['tweet_text'].apply(len)
data['video'] = data.media_type.eq('video')
data['photo'] = data.media_type.eq('photo')
data['num_urls'] = data['tweet_text'].str.count("http")-data['quote'].astype(int)+1 #quote gives link


scal = data.num_likes/data.Followers*100
data['target'] = 0
#target = scal.ge(0.092778723) #medium
#target = scal.ge(0.31737635) #good
#data.loc[scal.ge(0.092778723), 'target'] = 1#medium
#data.loc[scal.ge(0.31737635), 'target'] = 2#good

data.loc[scal.ge(0.043159258), 'target'] = 1#medium 33
data.loc[scal.ge(0.210837023) , 'target'] = 2#good 33


feature_cols = ['reply', 'quote', 'video', 'photo', 'num_hashtags', 'num_mentions', 'num_urls', 'num_emojis', 'emoji_polarity', 'subjectivity', 'length'] 
#feature_cols = ['video', 'photo', 'reply', 'quote', 'length', 'subjectivity', 'length'] 
# you want all rows, and the feature_cols' columns
X = data.loc[:, feature_cols]

X['polarity'] = (data['polarity']+1)/2

X['pvdabelgie'] = data['Screen Name'].eq("pvdabelgie")  | data['Screen Name'].eq("peter_mertens") | data['Screen Name'].eq("RaoulHedebouw") | data['Screen Name'].eq("TomDMeester") | data['Screen Name'].eq("JosDHaese") | data['Screen Name'].eq("LiseVDC") | data['Screen Name'].eq("DeWitteKim")
X['groen'] = data['Screen Name'].eq("groen") | data['Screen Name'].eq("MeyremAlmaci") | data['Screen Name'].eq("pdsutter") | data['Screen Name'].eq("kristofcalvo") | data['Screen Name'].eq("WouterDeVriendt") | data['Screen Name'].eq("BjornRzoska") | data['Screen Name'].eq("TinneVdS")
X['sp_a'] = data['Screen Name'].eq("sp_a") | data['Screen Name'].eq("johncrombez") | data['Screen Name'].eq("kvanbrempt") | data['Screen Name'].eq("conner_rousseau") | data['Screen Name'].eq("freyabos") | data['Screen Name'].eq("JorisVDBroucke") | data['Screen Name'].eq("MohamedRidouani")
X['cdenv'] = data['Screen Name'].eq("cdenv") |  data['Screen Name'].eq("joachimcoens") | data['Screen Name'].eq("wbeke") | data['Screen Name'].eq("Koen_Geens1") | data['Screen Name'].eq("crevits") | data['Screen Name'].eq("dalle_benjamin") | data['Screen Name'].eq("ServaisV")
X['openvld'] = data['Screen Name'].eq("openvld") | data['Screen Name'].eq("RuttenGwendolyn") | data['Screen Name'].eq("Maggie_DeBlock") | data['Screen Name'].eq("alexanderdecroo") | data['Screen Name'].eq("Barttommelein") | data['Screen Name'].eq("svengatz") | data['Screen Name'].eq("goedeleliekens")
X['de_NVA'] = data['Screen Name'].eq("de_NVA") | data['Screen Name'].eq("Bart_DeWever") | data['Screen Name'].eq("Zu_Demir") | data['Screen Name'].eq("FranckenTheo") | data['Screen Name'].eq("JanJambon") | data['Screen Name'].eq("LiesbethHomans") | data['Screen Name'].eq("jvanovertveldt")
X['vlbelang'] = data['Screen Name'].eq("vlbelang") | data['Screen Name'].eq("FDW_VB") | data['Screen Name'].eq("gannemans") | data['Screen Name'].eq("DVanLangenhove") | data['Screen Name'].eq("SamvanRooy1") | data['Screen Name'].eq("Barbara_Pas") | data['Screen Name'].eq("chrisjanssensVB") | data['Screen Name'].eq("FDW_VB, ")


time_tweet = pd.to_datetime(data['Time'], format='%Y-%m-%d %H:%M:%S')
delta = time_tweet.dt.date-date.today() 
X['days_since_post'] = -delta.dt.days/(7*30)

X['time_day'] = np.sin(2*np.pi*time_tweet.dt.hour/24)+1
X['day_week'] = np.sin(2*np.pi*time_tweet.dt.dayofweek/6)+1
X['hours'] = time_tweet.dt.hour/24

if new_version == True:
    X['num_hashtags_trending'] = data['num_hashtags_trending']

X['days_since_post'] = scaler.fit_transform(X.days_since_post.values.reshape(-1, 1))
X['length'] = scaler.fit_transform(X.length.values.reshape(-1, 1))
X['num_urls'] = scaler.fit_transform(X.num_urls.values.reshape(-1, 1))
X['num_hashtags'] = scaler.fit_transform(X.num_hashtags.values.reshape(-1, 1))
X['num_mentions'] = scaler.fit_transform(X.num_mentions.values.reshape(-1, 1))
X['num_emojis'] = scaler.fit_transform(X.num_emojis.values.reshape(-1, 1))
X['emoji_polarity'] = scaler.fit_transform(X.emoji_polarity.values.reshape(-1, 1))
X['polarity'] = scaler.fit_transform(X.polarity.values.reshape(-1, 1))


X.head(10)

,reply,quote,video,photo,num_hashtags,num_mentions,num_urls,num_emojis,emoji_polarity,subjectivity,...,sp_a,cdenv,openvld,de_NVA,vlbelang,days_since_post,time_day,day_week,hours,num_hashtags_trending
0,False,False,False,False,0.000000,0.090909,0.666667,0.0,0.317846,0.25,...,False,False,False,False,False,0.000000,1.500000,0.133975,0.416667,0
1,False,False,False,False,0.000000,0.000000,0.666667,0.0,0.317846,0.00,...,False,False,False,False,False,0.012146,0.133975,1.866025,0.666667,0
2,False,False,False,False,0.000000,0.000000,0.666667,0.0,0.317846,0.00,...,False,False,False,False,False,0.032389,2.000000,0.133975,0.250000,0
3,False,False,False,False,0.000000,0.000000,0.666667,0.0,0.317846,0.00,...,False,False,False,False,False,0.036437,1.258819,1.000000,0.458333,0
4,False,False,False,False,0.000000,0.000000,0.666667,0.0,0.317846,0.00,...,False,False,False,False,False,0.044534,1.707107,1.866025,0.375000,0
5,False,False,False,True,0.222222,0.000000,0.666667,0.0,0.317846,0.25,...,False,False,False,False,False,0.048583,1.258819,1.000000,0.458333,0
6,False,False,False,False,0.000000,0.000000,0.666667,0.0,0.317846,0.60,...,False,False,False,False,False,0.048583,1.500000,1.000000,0.416667,0
7,True,True,False,True,0.444444,0.181818,0.333333,0.0,0.317846,0.00,...,False,False,False,False,False,0.064777,0.292893,1.000000,0.625000,0
8,True,True,False,True,0.222222,0.090909,0.333333,0.0,0.317846,0.70,...,False,False,False,False,False,0.064777,0.292893,1.000000,0.625000,0
9,True,True,False,True,0.222222,0.181818,0.333333,0.0,0.317846,0.50,...,False,False,False,False,False,0.064777,0.292893,1.000000,0.625000,0


In [6]:
#X = pd.get_dummies(Xin,drop_first=True)

In [7]:
# Randomly split off a test set of approximatly 25% of the data

test_fraction = 0.25
x_train, x_test, r_train, r_test = train_test_split(X, data['target'], 
                                                    test_size = test_fraction, random_state=0)

#print(x_train)
#print(r_train)

# 2. Pipeline

Hyperparameter tuning: regularisation parameter C

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
def tune(x, r):
    pipe = Pipeline([
        ('scale',RobustScaler(quantile_range=(25, 75))),
        ('logreg',LogisticRegression(C=1.0, class_weight='balanced', solver='liblinear'))
    ])

    # Set the parameters by cross-validation
    C = [{'logreg__C': [1.0e-5,1.0e-4,1.0e-3,1.0e-2,1.0e-1,1.0, 
                                       10.0, 100.0, 1000.0,10000.0,100000]}]

    CV = GridSearchCV(pipe, C, cv=5)
    CV.fit(x, r)
    best_C = CV.best_params_['logreg__C']
    print("Optimal regularisation value: ",best_C)
    optimal_pipe = Pipeline([
        ('scale', RobustScaler(quantile_range=(25, 75))),
        ('logreg', LogisticRegression(C = best_C, class_weight='balanced', solver='liblinear'))
    ])
    
    return optimal_pipe

model = tune(x_train, r_train)
scores = cross_val_score(model, x_train, r_train, cv=5)
print(scores)
print("Average accuracy: ",scores.mean(),", stdev: ",scores.std())

/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/a

/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/a

Optimal regularisation value:  0.1
[0.61168385 0.64137931 0.6137931  0.59861592 0.59861592]
Average accuracy:  0.6128176193000778 , stdev:  0.015629356347640164


/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/a

# 3. Feature selection

In [9]:
num_features =len(X.columns)
scores = np.zeros(num_features)

for el in range(num_features):
    # Select k best features based on training data
    selector = SelectKBest(chi2, k=el+1).fit(x_train, r_train)
    # extract these features from training data
    x_new=selector.transform(x_train)
    # build and CV logistic regression model with these features
    s_model = tune(x_new, r_train)
    scores[el] = cross_val_score(s_model, x_new, r_train, cv=5).mean()
    print("Average accuracy with ",el+1," features: ",scores[el])

/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/a

Optimal regularisation value:  0.0001
Average accuracy with  1  features:  0.5258800403957405


/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/a

/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/a

Optimal regularisation value:  0.1
Average accuracy with  2  features:  0.5521250857466427


...

Optimal regularisation value:  0.1
Average accuracy with  23  features:  0.6135072744724915


/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/a

/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/a

Optimal regularisation value:  0.1
Average accuracy with  24  features:  0.6128176193000778


/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [10]:
best_features = np.argmax(scores)

# 4. Final model

In [11]:
select = SelectKBest(chi2, k=best_features+1).fit(x_train, r_train)
select_model = tune(selector.transform(x_train), r_train)
scoreslin = cross_val_score(select_model, selector.transform(x_train), r_train, cv=5)
print(scoreslin)
print("Average CV accuracy of linear model: ",scoreslin.mean(),", stdev: ",scoreslin.std())
select_model.fit(selector.transform(x_train), r_train)
y_pred = select_model.predict(selector.transform(x_test))

train_score = select_model.score(selector.transform(x_train), r_train)
test_score = select_model.score(selector.transform(x_test), r_test)

print("Accuracy of final model: ",train_score,
      " (train), ",test_score," (test)")

/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/a

/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/a

Optimal regularisation value:  0.1
[0.61168385 0.64137931 0.6137931  0.59861592 0.59861592]
Average CV accuracy of linear model:  0.6128176193000778 , stdev:  0.015629356347640164
Accuracy of final model:  0.6363008971704623  (train),  0.6542443064182195  (test)


/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/amdg/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [12]:
t = selector.transform(x_test)
print(selector)

SelectKBest(k=24, score_func=<function chi2 at 0x7f0bb559d560>)


In [13]:
#precision = sklearn.metrics.precision_score(r_test, y_pred)
confusion = sklearn.metrics.confusion_matrix(r_test, y_pred)
accuracy = sklearn.metrics.accuracy_score(r_test, y_pred)

#print("Precision of final model: ", precision)
#print("Accuracy2 of final model: ", accuracy)
print(accuracy)
print(confusion)

0.6542443064182195
[[113  32  13]
 [ 44  68  38]
 [ 11  29 135]]
